link to tuto: https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891

In [1]:
import os
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam


In [2]:
MAX_LEN = 64
dataset_path =  'D://data_phd_code_from_scatch//datasets//cornell movie-dialogs corpus'
### loading all data into memory
corpus_movie_conv = dataset_path +'/movie_conversations.txt'
corpus_movie_lines = dataset_path +'/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

In [3]:
### splitting text using special lines
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

### generate question answer pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()  
        second = lines_dic[ids[i+1]].strip() 

        qa_pairs.append(' '.join(first.split()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)

# train tokenizer

In [4]:
# WordPiece tokenization
### save data as txt file

text_data = []
file_count = 0
save_path_data_processed = 'D:/data_phd_code_from_scatch/datasets/data_processed/movie-dialogs'
for sample in tqdm.tqdm([x[0] for x in pairs]):
    text_data.append(sample)
    # once we hit the 10K mark, save to file
    if len(text_data) == 10000:
        with open(save_path_data_processed + '/text_'+ str(file_count) +'.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 221616/221616 [00:00<00:00, 1878112.59it/s]


In [7]:
# Specify the directory path

# List to hold the paths of .txt files
txt_files = []

# Walk through the directory
for root, dirs, files in os.walk(save_path_data_processed):
    for file in files:
        if file.endswith('.txt'):
            txt_files.append(os.path.join(root, file))

In [8]:
from scripts import tokenizer_train_bert
tokenizer = tokenizer_train_bert.train_bert_tokenizer(txt_files, 'bert_toy')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './bert_tokenizer'

# start

In [9]:
from transformers import BertTokenizer
vocab_file_path = 'bert_tokenizer/bert_toy-vocab.txt'
tokenizer = BertTokenizer(vocab_file_path)

In [ ]:
encoded_text = tokenizer.encode("Hello, how are you?", add_special_tokens=True)
decoded_text = tokenizer.decode(encoded_text)

print(f"Encoded Text: {encoded_text}")
print(f"Decoded Text: {decoded_text}")

# load dataset

In [10]:
from data.dataset_bert import BERTDataset
train_data = BERTDataset(pairs, tokenizer, seq_len=64)
train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)

In [11]:
sample_data = next(iter(train_loader))
print(train_data[random.randrange(len(train_data))])

{'bert_input': tensor([    1,   335,   332,    15,  1780,    15,     3,   217, 10062,   107,
          178,   248,     3,  9766,  9010,    17,    48,  2074,   384,  1506,
            3,   150, 12949,    17,   218, 12451,     3,   150,  7110, 11399,
           17,     2,  1272,    11,    58,   150, 18429,    17,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'bert_label': tensor([   0,    0,    0,    0,    0,    0,   48,    0,    0,    0,    0,    0,
         348,    0,    0,    0,    0,    0,    0,    0, 2610,    0,    0,    0,
           0,    0,  194,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), 'segment_label': tensor([1, 1, 

# pretrain

In [12]:
from data.dataset_bert import BERTDataset
from models.model_bert import BERT, BERTLM
from scripts.trainers_bert import BERTTrainer

train_data = BERTDataset(pairs, seq_len=MAX_LEN, tokenizer=tokenizer)
train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)

bert_model = BERT(
  vocab_size=len(tokenizer.vocab),
  d_model=768,
  n_layers=2,
  heads=12,
  dropout=0.1
)

bert_lm = BERTLM(bert_model, len(tokenizer.vocab))

bert_trainer = BERTTrainer(bert_lm, train_loader, device='cpu')
epochs = 20

for epoch in range(epochs):
  bert_trainer.train(epoch)


Total Parameters: 46699434


EP_train:0:   0%|| 1/6926 [00:03<5:59:48,  3.12s/it]

{'epoch': 0, 'iter': 0, 'avg_loss': 10.624918937683105, 'avg_acc': 31.25, 'loss': 10.624918937683105}


EP_train:0:   0%|| 1/6926 [00:06<11:49:24,  6.15s/it]


KeyboardInterrupt: 

In [14]:
bert_lm

BERTLM(
  (bert): BERT(
    (embedding): BERTEmbedding(
      (token): Embedding(21160, 768, padding_idx=0)
      (segment): Embedding(3, 768, padding_idx=0)
      (position): PositionalEmbedding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder_blocks): ModuleList(
      (0-1): 2 x EncoderLayer(
        (layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_multihead): MultiHeadedAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (output_linear): Linear(in_features=768, out_features=768, bias=True)
        )
        (feedforward): FeedForward(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout